In [4]:
import os
import json
import pickle as pkl
from collections import defaultdict, Counter
import collections

import numpy as np
import pytrec_eval
from tqdm import tqdm

import argparse
import scipy.stats
import sys
import pytrec_eval

In [21]:
def LSI_MAP(scores, topic_n):
    MAP = 0
    for qid in scores[topic_n]:
        MAP += scores[topic_n][str(qid)]['map']
    return MAP

def LSI_nDCG(scores, topic_n):
    ndcg = 0
    for qid in scores[topic_n]:
        ndcg += scores[topic_n][str(qid)]['ndcg']
    return ndcg
    
def JSON_results(file_name):
    with open(file_name) as f:
        data = json.load(f)
    return(data)
    
def t_test(first, second):
    measures = ["map", "ndcg"]
    query_ids = first.keys()
    first_scores = [first[qid][meas] for qid in query_ids for meas in measures]
    second_scores = [second[qid][meas] for qid in query_ids for meas in measures]
    return scipy.stats.ttest_rel(first_scores, second_scores)

def dict_results():
    return collections.defaultdict(dict_results)
    

In [22]:
# load LSI results
result_path = "./LSI_results"
if os.path.exists(result_path):
    with open(result_path, "rb") as reader:
        LSI_results = pkl.load(reader)
        
topics = [10, 50, 100, 500, 1000, 2000, 5000, 10000]
lsi_bow_val = {i: LSI_results["bow"][i]["val"] for i in topics}
lsi_bow_test = {i: LSI_results["bow"][i]["test"] for i in topics}
lsi_tfidf_val = {i: LSI_results["tfidf"][i]["val"] for i in topics}
lsi_tfidf_test = {i: LSI_results["tfidf"][i]["test"] for i in topics}

In [23]:
# AQ 4.1 get MAP / nDCG scores for LSI standard
lsi_bow_val_score = (LSI_MAP(lsi_bow_val, 500), LSI_nDCG(lsi_bow_val, 500))
lsi_bow_all_score = (LSI_MAP(lsi_bow_val, 500) + LSI_MAP(lsi_bow_val, 500), 
                     LSI_nDCG(lsi_bow_test, 500) + LSI_nDCG(lsi_bow_test, 500))

lsi_tfidf_val_score = (LSI_MAP(lsi_tfidf_val, 500), LSI_nDCG(lsi_tfidf_val, 500))
lsi_tfidf_all_score = (LSI_MAP(lsi_tfidf_val, 500) + LSI_MAP(lsi_tfidf_val, 500), 
                       LSI_nDCG(lsi_tfidf_test, 500) + LSI_nDCG(lsi_tfidf_test, 500))

print(lsi_bow_val_score)
print(lsi_bow_all_score)
print(lsi_tfidf_val_score)
print(lsi_tfidf_all_score)


(1.5426351702819414, 10.672523867529197)
(3.085270340563883, 115.47097494393682)
(2.8371663760950443, 12.494818836140174)
(5.674332752190089, 138.3587529651074)


In [24]:
# AQ 4.2 get t-test scores from base models 
# t-test input expects output from evaluator.evaluate(overall_ser)
# which should be what saved in the json file, use JSON_results(file_name)

tf_idf = JSON_results("tf-idf.json")
#word2vec = JSON_results("word2vec.json")
#doc2vec = JSON_results("doc2vec.json")
lsi_bow = {**lsi_bow_val[500], **lsi_bow_test[500]}
lsi_tfidf = {**lsi_tfidf_val[500], **lsi_tfidf_test[500]}
lda = JSON_results("lda.json")


t_test(tf_idf, lsi_bow)




    

Ttest_relResult(statistic=9.90028625507743, pvalue=3.721053378269528e-20)